In [ ]:
!pip install fastcoref
!pip install chromadb
!pip install gradio
!pip install spacy[transformers]
!git clone https://github.com/Ahmed-Khaled-Saleh/npat.git

## Vector Database

In [47]:
import json
with open('data/enhanced_economics_data_v1.json', 'r') as f:
    data = json.load(f)

In [48]:
data['data'][0].keys()

dict_keys(['authors', 'date_modify', 'description', 'filename', 'image_url', 'language', 'localpath', 'title', 'title_page', 'title_rss', 'source_domain', 'url', 'paragraphs', 'ID', 'linked_entites', 'related_persons'])

In [103]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.PersistentClient(
    path="db",
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

In [74]:
filtered_data = []
for article in data['data']:
    article_data = []
    for i, ent in enumerate(article['linked_entites']):
        quotes = []
        for j, p_ents in enumerate(ent):
            q = {}
            if p_ents:
                # print(f"Source: {p_ents['Speaker']}, cue: {p_ents['Cue']}, quote: {p_ents['Quote']}")
                quote_str = f"{p_ents['Speaker']} {p_ents['Cue']} {p_ents['Quote']}"
                q['Source'] = p_ents['Speaker']
                q['Cue'] = p_ents['Cue']
                q['Quote'] = p_ents['Quote']
                q['quote_str'] = quote_str
                quotes.append(q)
        if len(quotes) > 0:
            article_data.append({
                'paragraph_index': i,
                'quotes': quotes
            })
    filtered_data.append({
        'article_id': article['ID'],
        'title': article['title'],
        'url': article['url'],
        'quotes_data': article_data
    })
# with open('data/filtered_enhanced_economics_data_v1.json', 'w') as f:
#     json.dump({'data': filtered_data}, f)



In [ ]:
collection = client.get_or_create_collection("economics_quotes")
for article in filtered_data:
    try:
        collection.add(
            documents=[q['quote_str'] for para in article['quotes_data'] for q in para['quotes']],
            metadatas=[{"source": article['url']}]*sum([len(para['quotes']) for para in article['quotes_data']]),
            ids=[f"{article['article_id']}_quote_{i}" for i in range(sum([len(para['quotes']) for para in article['quotes_data']]))],
        )
    except Exception as e:
        print(f"Error adding quotes for article {article['article_id']}: {e}")

In [ ]:
from unsloth import FastLanguageModel
import torch

llm, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048,   
    load_in_4bit = True,     
    load_in_8bit = False,    
    full_finetuning = False, 
)

In [ ]:
from unsloth.chat_templates import get_chat_template
def get_response(relevant_quotes, user_query):
    
    sys_prompt = """You are a helpful assistant that extracts relevant quotations from a set of articles based on user queries:
    Given a user query, search the database of quotations and return the most relevant ones.
    Provide the quotations along with their sources in a clear format.
    If no relevant quotations are found, respond with "No relevant quotations found."
    Use the following format:
    Quotation: "<quote here>"
    Source: <source here>

    Here are the relevant quotations:
    {relevant_quotes}
    User Query: "{user_query}"
    Provide the relevant quotations below:
    """
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = "llama-3.1",
    )
    FastLanguageModel.for_inference(llm) 

    messages = [
        {"role": "user", "content": sys_prompt.format(relevant_quotes=relevant_quotes, user_query=user_query)},
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, 
        return_tensors = "pt",
    ).to("cuda")

    outputs = llm.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                            temperature = 1.5, min_p = 0.1)
    ans = tokenizer.batch_decode(outputs)
    return ans[0]

In [ ]:
import gradio as gr
import time

def respond(message, history):

    res = collection.query(
    query_texts=[message],
    n_results=2,
)
    bot_message = f"You said: {message}"
    ans = get_response(
        relevant_quotes = res,
        user_query = message,
    )
    
    
    return ans

with gr.ChatInterface(
    fn=respond, 
    title="Quotation Miner",
    textbox=gr.Textbox(placeholder="Ask me a question...", container=False, scale=7)
) as demo:

    demo.examples = [
        ["What did president Macron think about the coalition forces?"],
    ]

demo.launch()